# Load data

In [2]:
import os
import pandas as pd

data_path = "../data/results"
bart_ft_res = pd.read_csv(os.path.join(data_path,"bart-ft-results.csv"))
llama_ft_res = pd.read_csv(os.path.join(data_path,"llama-ft-results.csv"))
llama_instr_res = pd.read_csv(os.path.join(data_path,"llama-instruct-few-shot-results.csv"))

In [3]:
bart_ft_res

,dataset,base-b1,base-b2,masked-b1,masked-b2,subject-b1,subject-b2,target-phrase-b1,target-phrase-b2,target-sent-b1,target-sent-b2,target-sent-target-b1,target-sent-target-b2,target-sent-subject-b1,target-sent-subject-b2
0,c2s,20.127,6.451,18.792,4.865,30.405,19.002,21.887,9.306,22.931,11.731,23.814,12.907,31.640,20.938
1,c2sp,19.784,6.257,20.500,8.371,31.428,20.317,21.611,9.859,21.291,10.529,23.323,12.275,32.585,21.927
2,c4s,19.593,6.638,19.678,6.381,32.311,20.601,21.622,9.662,21.107,10.121,23.052,12.792,32.391,21.728
3,c4sp,18.544,6.585,20.353,7.890,30.628,19.846,20.773,10.017,21.272,10.893,24.881,13.791,32.564,21.634
4,cs,21.549,10.119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.099,10.541,30.854,19.994
5,c2spo,17.095,3.720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,c4spo,17.157,3.514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
